# BlackRoad Agent Communication Pattern Analysis

> **BlackRoad Labs** | Research notebook analyzing inter-agent communication patterns,
> task distribution, and memory chain integrity across the 30K agent fleet.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from datetime import datetime, timedelta
import hashlib, json, random

# Brand colors
COLORS = {
    'LUCIDIA': '#9400D3', 'ALICE': '#0047AB', 'OCTAVIA': '#FF4500',
    'PRISM': '#00CED1', 'ECHO': '#7B2FBE', 'CIPHER': '#444444',
}
plt.style.use('dark_background')
print('Libraries loaded ✓')

In [ ]:
# ── Synthetic agent communication log ────────────────────────────────────────
random.seed(42)
np.random.seed(42)

AGENTS = list(COLORS.keys())
TASK_TYPES = ['analysis', 'execution', 'memory', 'routing', 'security', 'inference']

# Generate 30 days of synthetic message logs
records = []
base = datetime(2026, 1, 1)
for i in range(10_000):
    sender = random.choice(AGENTS)
    receiver = random.choice([a for a in AGENTS if a != sender])
    records.append({
        'ts': base + timedelta(seconds=i * 259),
        'sender': sender,
        'receiver': receiver,
        'task_type': random.choice(TASK_TYPES),
        'latency_ms': int(np.random.lognormal(3, 1)),
        'success': random.random() > 0.03,
    })

df = pd.DataFrame(records)
df['hour'] = df['ts'].dt.hour
df['day'] = df['ts'].dt.day
print(f'Generated {len(df):,} communication events')
df.head()

In [ ]:
# ── Communication Heatmap ─────────────────────────────────────────────────────
matrix = pd.crosstab(df['sender'], df['receiver'])

fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(matrix.values, cmap='plasma', aspect='auto')
ax.set_xticks(range(len(AGENTS)))
ax.set_yticks(range(len(AGENTS)))
ax.set_xticklabels(matrix.columns, rotation=45, ha='right', fontsize=10)
ax.set_yticklabels(matrix.index, fontsize=10)
plt.colorbar(im, ax=ax, label='Message Count')
ax.set_title('Agent-to-Agent Communication Volume', fontsize=14, pad=12)
plt.tight_layout()
plt.savefig('figures/comm-heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ── P50/P95/P99 Latency by Agent ──────────────────────────────────────────────
latency = df.groupby('sender')['latency_ms'].quantile([0.5, 0.95, 0.99]).unstack()
latency.columns = ['p50', 'p95', 'p99']

fig, ax = plt.subplots(figsize=(9, 5))
x = np.arange(len(latency))
for i, (col, label) in enumerate(zip(['p50', 'p95', 'p99'], ['Median', 'p95', 'p99'])):
    ax.bar(x + i * 0.25, latency[col], 0.25, label=label)
ax.set_xticks(x + 0.25)
ax.set_xticklabels(latency.index)
ax.set_ylabel('Latency (ms)')
ax.set_title('Response Latency Distribution by Agent')
ax.legend()
plt.tight_layout()
plt.savefig('figures/latency.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ── PS-SHA∞ Memory Chain Integrity Check ─────────────────────────────────────
def ps_sha(prev_hash: str, content: str) -> str:
    import time
    return hashlib.sha256(f'{prev_hash}:{content}:{time.time_ns()}'.encode()).hexdigest()

# Build a demo chain
chain = []
prev = 'GENESIS'
events = ['Agent LUCIDIA initialized', 'Task #001 claimed', 'Task #001 completed',
          'Memory key "strategy" stored', 'Agent CIPHER security scan OK']
for event in events:
    h = ps_sha(prev, event)
    chain.append({'event': event, 'hash': h, 'prev': prev})
    prev = h

# Verify
ok = all(chain[i]['prev'] == (chain[i-1]['hash'] if i > 0 else 'GENESIS')
         for i in range(len(chain)))
print(f'Chain integrity: {"VALID ✓" if ok else "BROKEN ✗"}')
pd.DataFrame(chain)[['event','hash','prev']].assign(
    hash=lambda d: d['hash'].str[:16]+'...',
    prev=lambda d: d['prev'].str[:16]+'...'
)